In [1]:
import pandas
df = pandas.read_csv("goal_prediction_numeric_UPDATED.csv")

In [8]:
print(df.shape)

(1200, 44)


In [3]:
print(df.columns)

Index(['match_id', 'scored', 'fpl_minutes', 'fpl_goals', 'fpl_assits',
       'fpl_cs', 'fpl_points', 'fpl_bonus', 'fpl_cprice', 'player_rating',
       'touches', 'saves', 'total_pass', 'aerial_won', 'formation_place',
       'accurate_pass', 'total_tackle', 'aerial_lost', 'fouls', 'yellow_card',
       'total_scoring_att', 'man_of_the_match', 'goals', 'won_contest',
       'blocked_scoring_att', 'goal_assist', 'good_high_claim',
       'second_yellow', 'red_card', 'error_lead_to_goal', 'last_man_tackle',
       'six_yard_block', 'post_scoring_att', 'att_pen_target', 'penalty_save',
       'penalty_conceded', 'clearance_off_line', 'att_pen_goal',
       'att_pen_miss', 'own_goals', 'att_pen_post'],
      dtype='object')


In [5]:
print(df['scored'])

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
5       1.0
6       1.0
7       1.0
8       1.0
9       1.0
10      1.0
11      1.0
12      1.0
13      1.0
14      1.0
15      1.0
16      1.0
17      1.0
18      1.0
19      1.0
20      1.0
21      1.0
22      1.0
23      1.0
24      1.0
25      1.0
26      1.0
27      1.0
28      1.0
29      1.0
       ... 
1170    3.0
1171    3.0
1172    3.0
1173    3.0
1174    3.0
1175    3.0
1176    3.0
1177    3.0
1178    3.0
1179    3.0
1180    3.0
1181    3.0
1182    3.0
1183    3.0
1184    3.0
1185    3.0
1186    3.0
1187    3.0
1188    3.0
1189    3.0
1190    3.0
1191    3.0
1192    3.0
1193    3.0
1194    3.0
1195    3.0
1196    3.0
1197    3.0
1198    3.0
1199    3.0
Name: scored, Length: 1200, dtype: float64


In [3]:
score_1 = []
score_2 = []
score_3 = []

for i in df['scored']:
    if i == 1:
        score_1.append(1)
        score_2.append(0)
        score_3.append(0)
    elif i == 2:
        score_1.append(0)
        score_2.append(1)
        score_3.append(0)
    else:
        score_1.append(0)
        score_2.append(0)
        score_3.append(1)
        

In [4]:
df['score_1'] = score_1
df['score_2'] = score_2
df['score_3'] = score_3

In [48]:
#Torch Classes
#General Data processing, Linear Algebra and Plotting  classes

from __future__ import print_function, division
import os
import sys
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import struct
import math
from torch.utils import data
# Ignore warnings
import warnings
import csv
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

cuda = torch.device('cuda')

In [41]:
class Football(nn.Module):
    def __init__(self):
        super(Football,self).__init__()
        
        
        self.fc1 = nn.Linear(39,20)
        #self.fc2 = nn.Linear(50,20)
        self.fc3 = nn.Linear(20,5)
        #self.fc4 = nn.Linear(100,50)
        #self.fc5 = nn.Linear(50,20)
        self.softmax = nn.Linear(5,3)
        
    def forward(self,x,**kwargs):
        
        #print(x)
        x = F.leaky_relu(self.fc1(x))
        
        #x = F.leaky_relu(self.fc2(x))
       
        x = F.leaky_relu(self.fc3(x))
        
        #x = F.sigmoid(self.fc4(x))
        #x = F.sigmoid(self.fc5(x))
        
        x = F.sigmoid(self.softmax(x))
        # print(x)
   
        return x
        

In [63]:
import sys
def training_1(net, X,Y, criterion,optimizer,epoch=5):
    #palm_joints_list = [0:13]
    #phalanges_joints_list = []

    total_loss = 0

    for e in range(epoch):
        running_loss = 0.0
        print("Starting Epoch %d..."%(e+1))
        for i,(inputs,labels) in enumerate(zip(X,Y),0):
            
            inputs = torch.from_numpy(inputs)
            inputs = inputs.type(torch.FloatTensor)
        
            optimizer.zero_grad()
            y_pred = net(inputs)
            labels = torch.from_numpy(np.array(labels))
            #labels.requires_grad = False
            labels = labels.type(torch.FloatTensor)
            loss = criterion(y_pred,labels)
            
            loss.backward()
            optimizer.step()
        
        #printing stats 
            sys.stdout.write('\r [epoch: %d, iteration: %5d] loss: %.3f ' % (e+1,i+1,running_loss))
            sys.stdout.flush()
        
            running_loss += loss.item()
            total_loss += loss.item()**0.5
            
        
        torch.save(net,"models/football_2_%d.pth"%(e+1))
        torch.save(optimizer,"optim.pth")
        
        total_loss = 0
        print("\nEpoch %d complete. Model after epoch %d successfully saved as models/football_net_2_%d.pth..."%(e+1,e+1,e+1))
        
    print('Finished Training...')

In [64]:
criterion = nn.MSELoss()
football_net = Football()
print(football_net)
optimizer = optim.Adam(football_net.parameters(),weight_decay = float(0.0001),lr= float(0.001))

Football(
  (fc1): Linear(in_features=39, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=5, bias=True)
  (softmax): Linear(in_features=5, out_features=3, bias=True)
)


In [54]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,2:].values,df.iloc[:,1].values, test_size = 0.1, random_state = 1 , stratify = df.iloc[:,1].values)

In [76]:
training_1(football_net,X_train[:,:39],X_train[:,39:],criterion,optimizer,epoch = 10)

Starting Epoch 1...
 [epoch: 1, iteration:  1080] loss: 210.593 
Epoch 1 complete. Model after epoch 1 successfully saved as models/football_net_2_1.pth...
Starting Epoch 2...
 [epoch: 2, iteration:  1080] loss: 209.317 
Epoch 2 complete. Model after epoch 2 successfully saved as models/football_net_2_2.pth...
Starting Epoch 3...
 [epoch: 3, iteration:  1080] loss: 210.205 
Epoch 3 complete. Model after epoch 3 successfully saved as models/football_net_2_3.pth...
Starting Epoch 4...
 [epoch: 4, iteration:  1080] loss: 210.202 
Epoch 4 complete. Model after epoch 4 successfully saved as models/football_net_2_4.pth...
Starting Epoch 5...
 [epoch: 5, iteration:  1080] loss: 209.118 
Epoch 5 complete. Model after epoch 5 successfully saved as models/football_net_2_5.pth...
Starting Epoch 6...
 [epoch: 6, iteration:  1080] loss: 209.336 
Epoch 6 complete. Model after epoch 6 successfully saved as models/football_net_2_6.pth...
Starting Epoch 7...
 [epoch: 7, iteration:  1080] loss: 208.370 

In [77]:
net = torch.load("models/football_1_10.pth")

In [78]:
error = 0
#total_sum = 0
maxi,mini = -100,100
predicts = []
for row in df.values:
    x = row[2:41]
    x = torch.from_numpy(np.array(x))
    x = x.type(torch.FloatTensor)
    y = row[41:]
    pred = net(x).tolist()
    
    '''if max < pred:
        max = pred
    if min > pred:
        min = pred
    '''
    
    '''if pred > 0.5:
        pred=1
    else :
        pred=0'''
    
    #print(pred)
    max_ind = pred.index(max(pred))
   
    pred = [0,0,0]
    pred[max_ind] = 1
    
    #total_sum += pred 
    
    predicts.append(pred)
    error += (sum(abs(y-pred)))

In [79]:
predicts = np.array(predicts)

In [80]:
min(predicts.T[2])

1

In [75]:
print(sum(predicts.T[2]))

1200


In [ ]:
sum(dff.iloc[:,1])

In [ ]:
print(1-sum(abs(predicts - dff.iloc[:,1]))/len(dff))

In [23]:
confmat = confusion_matrix(y_true = df.iloc[:,41].tolist(),y_pred = predicts.T[0])

In [24]:
print(confmat)

[[800   0]
 [400   0]]
